In [497]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from scipy.spatial.distance import cdist
import pandas as pd
import pymysql
import datetime
import sys
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import sklearn.feature_extraction.text as text
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/karthikravi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [498]:
rds_host = 'xbias-backend.cytnlddqky1c.us-east-2.rds.amazonaws.com'
username = 'xBias_master'
password = 'Kr19972=14'
db_name = 'xbias_backend'

In [524]:
description_list=[]
yesterday = (datetime.date.today()-datetime.timedelta(1)).strftime('%Y-%m-%d')
yesterday="'%"+yesterday+"%'"

# print(today)
#  WHERE publish_timestamp like {0}

conn = pymysql.connect(rds_host, user = username, passwd = password, db= db_name, connect_timeout = 5)
with conn.cursor() as cur:
    cur.execute("""SELECT * FROM article_table WHERE publish_timestamp like {0}""".format(yesterday))
    result = cur.fetchall()
    cur.close()
    i=1
    for row in result:
        if row[5] != 'No Listed description':
            description_list.append(row[5])
        else:
            description_list.append(row[4])

description_list=np.array(description_list)
print(len(description_list))

189


In [501]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(description_list).toarray()
vocab_tfidf = np.array(tfidf_vectorizer.get_feature_names())

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer( stop_words='english', min_df=20)
tf = tf_vectorizer.fit_transform(description_list).toarray()
vocab_tf = np.array(tf_vectorizer.get_feature_names())
#print(tf_feature_names)

In [502]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 17

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvda')
doctopics=nmf.fit_transform(tfidf)
count= 0
topic_dic={}
for i in range(len(doctopics)):
    if (np.max(doctopics[i]) != 0):
        count+=1
        if np.argmax(doctopics[i]) not in topic_dic:
            topic_dic[np.argmax(doctopics[i])]=[]
        topic_dic[np.argmax(doctopics[i])].append(str(description_list[i]))
        print(str(np.argmax(doctopics[i])) + ": "+ str(description_list[i]))
# # Run LDA
# lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

2: Andrew Gillum, a Black Progressive, and Ron DeSantis, a Trump Acolyte, Win Florida Governor Primaries
1: Nearly 3000 people died in Puerto Rico in the wake of Hurricane Maria
16: California Lawmakers Set Goal for Carbon-Free Energy by 2045
2: Tallahassee Mayor Andrew Gillum will win the Democratic nomination for governor of Florida, CNN projects, scoring a major victory for the party's progressive wing while taking another step toward becoming the state's first black chief executive.
1: The official count had been 64 in the 11 months since the storm, though media outlets and academics put the count much higher, more than 1,000.
3: The legislation comes on top of an earlier move requiring the state to generate 50 percent of its electricity from renewable sources by 2030.
7: “They will end everything immediately,” the president told a meeting of evangelical leaders at the White House, according to an audiotape of the remarks.
12: With Mr. Gillum’s upset victory, the governor’s race wi

In [503]:
topic_words = []

for topic in nmf.components_:
    word_idx = np.argsort(topic)[::-1][0:20]
    topic_words.append([vocab_tfidf[i] for i in word_idx])

print(count)
print(str(count/len(description_list)*100))

140
74.07407407407408


In [493]:
for phrase in topic_dic[0]:
    print(phrase)
    print()

U.S. President Donald Trump said on Twitter early on Wednesday China hacked the emails of 2016 Democratic presidential candidate Hillary Clinton but did not offer any evidence or further information.

Trump Takes Battle Over Media Coverage to `Rigged' News Feeds

Legally, President Donald Trump could probably terminate the North American Free Trade Agreement., experts say. Politically, it won't be easy.

Donald Trump was the big winner Tuesday night

President Donald Trump took a huge risk by holding a summit with North Korean leader Kim Jong Un and hailing it as a stunning history-changing success.

Donald Trump Was Forced To Unblock Twitter Critics. Now They're Getting Sweet Revenge.

What happens when you Google ""Trump news"" from abroad.

Lawyers for former Trump chairman said the ‘inside-the-Beltway’ area is saturated with negative political news.

Critics feel CNN is facing a credibility crisis for standing by a report on the infamous 2016 Trump Tower meeting despite ex-Clinton 

In [504]:
for t in range(len(topic_words)):
    print("Topic {}: {}".format(t, ' '.join(topic_words[t][:15])))

Topic 0: hacked clinton chinese emails president hillary donald trump claimed server email claim spread fbi wednesday
Topic 1: maria hurricane rico puerto toll death storm people 2975 died deadliest 64 nearly 3000 climbed
Topic 2: florida major gillum andrew governor win democratic black republican progressive nomination lessons shocking era tallahassee
Topic 3: lying state began arizonans phoenix respects senator late pay capitol honor arizona people honoring marked
Topic 4: sound puget washington navy zone guard safety coast divers bay retrieve went unexploded port orchard
Topic 5: nixon cuomo debate cynthia square challenger york set new hotly anticipated andrew stop gov spar
Topic 6: culmination circle frustrations come widely departure seen crisis mcgahn new president trump coming exit experts
Topic 7: house white don mcgahn leaving counsel post yes big swing fall spin deal president plans
Topic 8: russian stay unpopular overhaul reduced addressing pension age increase men rise wo

In [417]:
doctopic[2, :]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [313]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {0}".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
# display_topics(lda, tf_feature_names, no_top_words)

How a Pentagon deal became an identity crisis for Google
